In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
from geopy.extra.rate_limiter import RateLimiter
from geopy.geocoders import Nominatim
import folium
from sklearn.cluster import KMeans ,AgglomerativeClustering
import gensim

In [2]:
#Read data
data = pd.read_excel('MSdata.xlsx')
#data.drop(['InvoiceId', 'InvoiceNumberFull'], axis=1, inplace=True)
print(data.head())

month  TermOfPaymentDays  Amount_netCN  DefaultStatus  \
0     12                110        204.59              0   
1     12                 52        134.79              0   
2     12                 30         13.89              0   
3     12                 30         19.98              0   
4     12                 30          9.99              0   

                     Email  Age    ZIP  Country  ProbDefault  ScoreValue  \
0        WHohlfeld@gmx.net   68  15890  Germany     0.026832       502.0   
1   Silvia_wagner95@web.de   23  49632  Germany     0.010632       576.0   
2                      NaN   67   2899  Germany     0.022672       515.0   
3  marinellabeyer@yahoo.de   51  48599  Germany     0.057552       472.0   
4                      NaN   67   6458  Germany     0.005112       645.0   

                    State  GDP by State  Unemployment rate  
0             Brandenburg       29400.0              0.057  
1     Lower Saxony-Anhalt       37100.0              0.051  
2 

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15125 entries, 0 to 15124
Data columns (total 13 columns):
month                15125 non-null int64
TermOfPaymentDays    15125 non-null int64
Amount_netCN         15125 non-null float64
DefaultStatus        15125 non-null int64
Email                4418 non-null object
Age                  15125 non-null int64
ZIP                  15125 non-null int64
Country              15125 non-null object
ProbDefault          14813 non-null float64
ScoreValue           15124 non-null float64
State                15082 non-null object
GDP by State         15082 non-null float64
Unemployment rate    15082 non-null float64
dtypes: float64(5), int64(5), object(3)
memory usage: 1.5+ MB


In [4]:
#Drop Null rows
data.dropna(subset=['State', 'ScoreValue' ,'ProbDefault'], inplace=True)
data.reset_index(drop=True)
#check null values again
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14771 entries, 0 to 15124
Data columns (total 13 columns):
month                14771 non-null int64
TermOfPaymentDays    14771 non-null int64
Amount_netCN         14771 non-null float64
DefaultStatus        14771 non-null int64
Email                4327 non-null object
Age                  14771 non-null int64
ZIP                  14771 non-null int64
Country              14771 non-null object
ProbDefault          14771 non-null float64
ScoreValue           14771 non-null float64
State                14771 non-null object
GDP by State         14771 non-null float64
Unemployment rate    14771 non-null float64
dtypes: float64(5), int64(5), object(3)
memory usage: 1.6+ MB


In [5]:
#Data with all needed features
Data_clean = data[['month', "Age","Amount_netCN", "TermOfPaymentDays", 'GDP by State', 'Unemployment rate', 'ZIP', "DefaultStatus" ]]
Data_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14771 entries, 0 to 15124
Data columns (total 8 columns):
month                14771 non-null int64
Age                  14771 non-null int64
Amount_netCN         14771 non-null float64
TermOfPaymentDays    14771 non-null int64
GDP by State         14771 non-null float64
Unemployment rate    14771 non-null float64
ZIP                  14771 non-null int64
DefaultStatus        14771 non-null int64
dtypes: float64(3), int64(5)
memory usage: 1.0 MB


In [6]:
# Print a null value column array
print(Data_clean.columns[Data_clean.isnull().any()])

Index([], dtype='object')


In [7]:
#Creating dummy variables
month = pd.get_dummies(Data_clean['month'])
len(month.columns)
name_month = []
for i in range(1,len(month.columns)+1):
    name_month.append('month'+str(i))
   
month.columns = name_month

In [8]:
#Concatenating the dummy variables to the original dataset 
Data_clean=pd.concat([Data_clean,month],axis=1)
Data_clean.head(10)

,month,Age,Amount_netCN,TermOfPaymentDays,GDP by State,Unemployment rate,ZIP,DefaultStatus,month1,month2,month3,month4,month5
0,12,68,204.59,110,29400.0,0.057,15890,0,0,0,0,0,1
1,12,23,134.79,52,37100.0,0.051,49632,0,0,0,0,0,1
2,12,67,13.89,30,31000.0,0.053,2899,0,0,0,0,0,1
3,12,51,19.98,30,39400.0,0.066,48599,0,0,0,0,0,1
4,12,67,9.99,30,28700.0,0.070,6458,0,0,0,0,0,1
5,12,53,59.90,30,46300.0,0.031,89604,0,0,0,0,0,1
6,12,66,9.99,30,27900.0,0.067,17493,0,0,0,0,0,1
7,12,66,9.90,30,28700.0,0.070,39615,0,0,0,0,0,1
8,12,63,59.90,30,27900.0,0.067,18195,0,0,0,0,0,1
9,12,78,9.90,30,46300.0,0.031,79244,0,0,0,0,0,1


In [9]:
Data_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14771 entries, 0 to 15124
Data columns (total 13 columns):
month                14771 non-null int64
Age                  14771 non-null int64
Amount_netCN         14771 non-null float64
TermOfPaymentDays    14771 non-null int64
GDP by State         14771 non-null float64
Unemployment rate    14771 non-null float64
ZIP                  14771 non-null int64
DefaultStatus        14771 non-null int64
month1               14771 non-null uint8
month2               14771 non-null uint8
month3               14771 non-null uint8
month4               14771 non-null uint8
month5               14771 non-null uint8
dtypes: float64(3), int64(5), uint8(5)
memory usage: 1.1 MB


In [10]:
Data_clean[Data_clean['ZIP'] =='50']

,month,Age,Amount_netCN,TermOfPaymentDays,GDP by State,Unemployment rate,ZIP,DefaultStatus,month1,month2,month3,month4,month5


In [11]:
# Data_clean['domain'] = Data_clean['Email'].str.split('@').str[1]
# Data_clean['domain'].head()
# Data_clean['domain'] = Data_clean['domain'].str.lower()

In [12]:
# df_email=Data_clean['domain'].value_counts(normalize=True)
# df_email.head()

In [13]:
# df_email_dict = df_email.to_dict() 

In [14]:
# Data_clean['Email_popularity'] = Data_clean['domain'].apply(lambda x: df_email_dict[x])
# #Data_clean.head()

In [15]:
# Create a cross table of the loan intent and status
ZipCode =pd.crosstab(Data_clean['ZIP'], Data_clean['DefaultStatus'], margins = True)
ZipCode

DefaultStatus,0,1,All
ZIP,,,
1067,2,0,2
1069,2,0,2
1097,3,0,3
1099,6,0,6
1108,1,0,1
...,...,...,...
99988,1,0,1
99991,2,0,2
99994,1,0,1


In [16]:
ZipCode['Feq_default_Country']=ZipCode[1]/ZipCode[1].sum()
ZipCode['Feq_default_ZipCode']=ZipCode[1]/ZipCode['All']
ZipCode.sort_values(by=['Feq_default_ZipCode'], ascending=False)

DefaultStatus,0,1,All,Feq_default_Country,Feq_default_ZipCode
ZIP,,,,,
17390,0,1,1,0.000552,1.0
34393,0,1,1,0.000552,1.0
92507,0,1,1,0.000552,1.0
22147,0,1,1,0.000552,1.0
34549,0,1,1,0.000552,1.0
...,...,...,...,...,...
37445,1,0,1,0.000000,0.0
37434,3,0,3,0.000000,0.0
37431,3,0,3,0.000000,0.0


In [17]:
ZipCode.drop(columns=[0, 1, 'All', 'Feq_default_Country'], inplace= True)

In [18]:
ZipCode.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5290 entries, 1067 to All
Data columns (total 1 columns):
Feq_default_ZipCode    5290 non-null float64
dtypes: float64(1)
memory usage: 82.7+ KB


In [19]:
Data_clean = pd.merge(Data_clean, ZipCode['Feq_default_ZipCode'], on='ZIP')
Data_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14771 entries, 0 to 14770
Data columns (total 14 columns):
month                  14771 non-null int64
Age                    14771 non-null int64
Amount_netCN           14771 non-null float64
TermOfPaymentDays      14771 non-null int64
GDP by State           14771 non-null float64
Unemployment rate      14771 non-null float64
ZIP                    14771 non-null object
DefaultStatus          14771 non-null int64
month1                 14771 non-null uint8
month2                 14771 non-null uint8
month3                 14771 non-null uint8
month4                 14771 non-null uint8
month5                 14771 non-null uint8
Feq_default_ZipCode    14771 non-null float64
dtypes: float64(4), int64(4), object(1), uint8(5)
memory usage: 1.2+ MB


In [20]:
Data_clean.head()

,month,Age,Amount_netCN,TermOfPaymentDays,GDP by State,Unemployment rate,ZIP,DefaultStatus,month1,month2,month3,month4,month5,Feq_default_ZipCode
0,12,68,204.59,110,29400.0,0.057,15890,0,0,0,0,0,1,0.0
1,11,61,41.87,139,29400.0,0.057,15890,0,0,0,0,1,0,0.0
2,11,61,249.50,30,29400.0,0.057,15890,0,0,0,0,1,0,0.0
3,10,64,58.78,30,29400.0,0.057,15890,0,0,0,1,0,0,0.0
4,10,85,80.67,30,29400.0,0.057,15890,0,0,0,1,0,0,0.0


In [21]:
Data_clean.drop(columns=['month', 'ZIP'], inplace=True)
Data_clean.head()

,Age,Amount_netCN,TermOfPaymentDays,GDP by State,Unemployment rate,DefaultStatus,month1,month2,month3,month4,month5,Feq_default_ZipCode
0,68,204.59,110,29400.0,0.057,0,0,0,0,0,1,0.0
1,61,41.87,139,29400.0,0.057,0,0,0,0,1,0,0.0
2,61,249.50,30,29400.0,0.057,0,0,0,0,1,0,0.0
3,64,58.78,30,29400.0,0.057,0,0,0,1,0,0,0.0
4,85,80.67,30,29400.0,0.057,0,0,0,1,0,0,0.0


In [22]:
# Data_clean.to_excel('Data_clean_noGeo.xlsx', index = False)

In [23]:
# #Read data
# Data_clean = pd.read_excel('Data_clean_noGeo.xlsx')
# Data_clean.head()

In [24]:
Data_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14771 entries, 0 to 14770
Data columns (total 12 columns):
Age                    14771 non-null int64
Amount_netCN           14771 non-null float64
TermOfPaymentDays      14771 non-null int64
GDP by State           14771 non-null float64
Unemployment rate      14771 non-null float64
DefaultStatus          14771 non-null int64
month1                 14771 non-null uint8
month2                 14771 non-null uint8
month3                 14771 non-null uint8
month4                 14771 non-null uint8
month5                 14771 non-null uint8
Feq_default_ZipCode    14771 non-null float64
dtypes: float64(4), int64(3), uint8(5)
memory usage: 995.3 KB


In [25]:
# Check the values created by the predict method
print(Data_clean['DefaultStatus'].value_counts())

0    13865
1      906
Name: DefaultStatus, dtype: int64


In [26]:
count_nondefault, count_default = Data_clean['DefaultStatus'].value_counts()

In [27]:
count_nondefault

13865

In [28]:
# Create data sets for defaults and non-defaults
nondefaults = Data_clean [Data_clean ['DefaultStatus'] == 0]
defaults = Data_clean [Data_clean ['DefaultStatus'] == 1]

In [29]:
nondefaults

,Age,Amount_netCN,TermOfPaymentDays,GDP by State,Unemployment rate,DefaultStatus,month1,month2,month3,month4,month5,Feq_default_ZipCode
0,68,204.59,110,29400.0,0.057,0,0,0,0,0,1,0.0
1,61,41.87,139,29400.0,0.057,0,0,0,0,1,0,0.0
2,61,249.50,30,29400.0,0.057,0,0,0,0,1,0,0.0
3,64,58.78,30,29400.0,0.057,0,0,0,1,0,0,0.0
4,85,80.67,30,29400.0,0.057,0,0,0,1,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
14766,47,150.67,30,47900.0,0.027,0,1,0,0,0,0,0.0
14767,63,154.57,30,47900.0,0.027,0,1,0,0,0,0,0.0
14768,58,130.77,30,29700.0,0.052,0,1,0,0,0,0,0.0
14769,64,130.77,30,46300.0,0.031,0,1,0,0,0,0,0.0


In [30]:
# Undersample the non-defaults
nondefaults_under = nondefaults.sample(count_default)

# Concatenate the undersampled nondefaults with defaults
data_balanced = pd.concat([nondefaults_under.reset_index(drop = True),
                             defaults.reset_index(drop = True)], axis = 0)

# Print the value counts for loan status
print(data_balanced['DefaultStatus'].value_counts())

1    906
0    906
Name: DefaultStatus, dtype: int64


In [31]:
data_balanced.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1812 entries, 0 to 905
Data columns (total 12 columns):
Age                    1812 non-null int64
Amount_netCN           1812 non-null float64
TermOfPaymentDays      1812 non-null int64
GDP by State           1812 non-null float64
Unemployment rate      1812 non-null float64
DefaultStatus          1812 non-null int64
month1                 1812 non-null uint8
month2                 1812 non-null uint8
month3                 1812 non-null uint8
month4                 1812 non-null uint8
month5                 1812 non-null uint8
Feq_default_ZipCode    1812 non-null float64
dtypes: float64(4), int64(3), uint8(5)
memory usage: 122.1 KB


In [32]:
data_balanced.head()

,Age,Amount_netCN,TermOfPaymentDays,GDP by State,Unemployment rate,DefaultStatus,month1,month2,month3,month4,month5,Feq_default_ZipCode
0,18,40.88,30,46300.0,0.031,0,1,0,0,0,0,0.000000
1,75,70.87,30,27900.0,0.067,0,0,1,0,0,0,0.000000
2,34,60.78,30,37100.0,0.051,0,0,0,0,1,0,0.000000
3,60,47.88,30,28700.0,0.070,0,0,1,0,0,0,0.090909
4,71,50.88,30,33600.0,0.051,0,0,1,0,0,0,0.000000


In [33]:
data_balanced.to_excel('data_balanced.xlsx', index = False)